<a href="https://colab.research.google.com/github/puruyadav-tech/ml-demo/blob/main/Copy_of_Untitled19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok --quiet

from google.colab import files
uploaded = files.upload()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 499.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.1 MB/s eta 0:00:00


Saving mdl.pkl to mdl.pkl


In [4]:
%%writefile app.py
import streamlit as st
import pickle
import pandas as pd
from PIL import Image
import os

pipe = pickle.load(open('mdl.pkl', 'rb'))

teams = [
    'Sunrisers Hyderabad', 'Mumbai Indians', 'Royal Challengers Bangalore',
    'Kolkata Knight Riders', 'Kings XI Punjab', 'Chennai Super Kings',
    'Rajasthan Royals', 'Delhi Capitals'
]

cities = [
    'Hyderabad', 'Bangalore', 'Mumbai', 'Indore', 'Kolkata', 'Delhi',
    'Chandigarh', 'Jaipur', 'Chennai', 'Cape Town', 'Port Elizabeth',
    'Durban', 'Centurion', 'East London', 'Johannesburg', 'Kimberley',
    'Bloemfontein', 'Ahmedabad', 'Cuttack', 'Nagpur', 'Dharamsala',
    'Visakhapatnam', 'Pune', 'Raipur', 'Ranchi', 'Abu Dhabi',
    'Sharjah', 'Mohali', 'Bengaluru'
]

logo_urls = {
    "Chennai Super Kings": "https://upload.wikimedia.org/wikipedia/en/2/2d/Chennai_Super_Kings_Logo.png",
    "Delhi Capitals": "https://upload.wikimedia.org/wikipedia/en/d/dc/Delhi_Capitals.svg",
    "Kings XI Punjab": "https://upload.wikimedia.org/wikipedia/en/d/d4/Kings_XI_Punjab_Logo.svg",
    "Kolkata Knight Riders": "https://upload.wikimedia.org/wikipedia/en/4/4f/Kolkata_Knight_Riders_Logo.svg",
    "Mumbai Indians": "https://upload.wikimedia.org/wikipedia/en/2/25/Mumbai_Indians_Logo.svg",
    "Rajasthan Royals": "https://upload.wikimedia.org/wikipedia/en/6/60/Rajasthan_Royals_Logo.svg",
    "Royal Challengers Bangalore": "https://upload.wikimedia.org/wikipedia/en/4/4c/Royal_Challengers_Bangalore_Logo.svg",
    "Sunrisers Hyderabad": "https://upload.wikimedia.org/wikipedia/en/8/81/Sunrisers_Hyderabad_Logo.svg"
}

st.title('🏏 IPL Win Predictor')

col1, col2 = st.columns(2)
with col1:
    batting_team = st.selectbox('Select Batting Team', sorted(teams))
with col2:
    bowling_team = st.selectbox('Select Bowling Team', sorted(teams))

if batting_team in logo_urls:
    st.image(logo_urls[batting_team], width=100, caption="Batting Team")
if bowling_team in logo_urls:
    st.image(logo_urls[bowling_team], width=100, caption="Bowling Team")

selected_city = st.selectbox('Select Host City', sorted(cities))
target = st.number_input('Target Score', step=1)

col3, col4, col5 = st.columns(3)
with col3:
    score = st.number_input('Current Score', step=1)
with col4:
    overs = st.number_input('Overs Completed', step=0.1)
with col5:
    wickets = st.number_input('Wickets Fallen', step=1)

if st.button('Predict Probability'):
    try:
        runs_left = target - score
        balls_left = 120 - int(overs * 6)
        wickets_remaining = 10 - wickets
        crr = score / overs if overs > 0 else 0
        rrr = (runs_left * 6) / balls_left if balls_left > 0 else 0

        input_df = pd.DataFrame({
            'batting_team': [batting_team],
            'bowling_team': [bowling_team],
            'city': [selected_city],
            'runs_left': [runs_left],
            'balls_left': [balls_left],
            'wickets': [wickets_remaining],
            'total_runs_x': [target],
            'crr': [crr],
            'rrr': [rrr]
        })

        result = pipe.predict_proba(input_df)
        loss = result[0][0]
        win = result[0][1]

        st.subheader(f"✅ {batting_team} - {round(win * 100)}% chance to win")
        st.subheader(f"❌ {bowling_team} - {round(loss * 100)}% chance to win")

    except Exception as e:
        st.error(f"⚠️ Error: {e}")


Overwriting app.py


In [5]:
!streamlit run app.py &>/content/log.txt &


In [6]:
from pyngrok import ngrok

# Optional: Only run this once
!ngrok config add-authtoken YOUR_AUTHTOKEN_HERE

# Start ngrok tunnel
public_url = ngrok.connect(8501)
print(f"🌐 Your app is live at: {public_url}")


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🌐 Your app is live at: NgrokTunnel: "https://8477-35-234-2-53.ngrok-free.app" -> "http://localhost:8501"
